# Answers!

## Task! Get some data

With the filters detect_isPrimary, and refExtendedness ==1 (galaxies only), get the following quantities in the g band:

* From the DP0.2 schema look for the PSF shapes (use PSF rather than DebiasedPSF)
* Compute the trace of the PSF (ixx + iyy)
* Look for cModel magnitudes, and read in the g band magnitudes (you'll need to convert from fluxes)
* Get RA/ DEC

Hints:
* Check utils.py for a flux to magnitude conversion function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
sys.path.insert(0,'/global/cfs/projectdirs/lsst/groups/SRV/gcr-catalogs-test')
import GCRCatalogs

%matplotlib inline


In [ ]:
catalog = GCRCatalogs.load_catalog('lsst_object')

I'm going to cheat a little from inspecting the schema, but here are the quantities we're looking for!

In [ ]:
quantity_list = sorted(catalog.list_all_native_quantities())
for i in range(len(quantity_list)):
    q = quantity_list[i]
    info = catalog.get_quantity_info(q)
    if ('g_' in q) and ('PSF' in q) and 'Debiased' not in q:
        print(q)



Looks like we need to add the flag too! It's already in your filter_list if you want to just add the whole thing in. Let's look for cModelFlux values... Looks like they're simply g_cModelFlux etc.

In [ ]:
quantity_list = sorted(catalog.list_all_native_quantities())
for i in range(len(quantity_list)):
    q = quantity_list[i]
    info = catalog.get_quantity_info(q)
    if ('g_' in q) and ('cModel' in q):
        print(q)


Now let's get our functions:


In [ ]:
def trace(ixx,ixy,iyy):
    return (ixx+iyy)


# borrowed from GCRCatalogs, can get this as from GCRCatalogs.dc2dm_catalog import convert_nanoJansky_to_mag, it's also in utils.py
def convert_nanoJansky_to_mag(flux):
    """Convert calibrated nanoJansky flux to AB mag.
    """
    AB_mag_zp_wrt_Jansky = 8.90  # Definition of AB
    # 9 is from nano=10**(-9)
    AB_mag_zp_wrt_nanoJansky = 2.5 * 9 + AB_mag_zp_wrt_Jansky

    return -2.5 * np.log10(flux) + AB_mag_zp_wrt_nanoJansky


In [ ]:
quantities = ["g_ra","g_decl","g_ixxPSF", "g_ixyPSF", "g_iyyPSF","g_cModelFlux"]
filters = ['detect_isPrimary','g_extendedness==1', 'g_iPSF_flag==False', 'g_cModel_flag==False']

data = catalog.get_quantities(quantities, filters =filters)  

plt.figure()
plt.hist2d(data["g_ra"],data["g_decl"],bins=1000)
plt.show()

In [ ]:
#add the extra quantities
data['T_PSF'] = trace(data['g_ixxPSF'],data['g_ixyPSF'],data['g_iyyPSF'])
data['g_mag'] = convert_nanoJansky_to_mag(data['g_cModelFlux'])

Let's check it looks okay!

In [ ]:
from scipy.stats import binned_statistic_2d as bs2d
plt.title('g-band PSF size variation')
val, xedge, yedge, bincount = bs2d(data["g_ra"],data["g_decl"],data['T_PSF'],statistic='median',bins=100)
plt.imshow(val,extent=[xedge[0],xedge[-1],yedge[0],yedge[-1]])
plt.colorbar()
plt.show()
